In [1]:
%reset -f

In [2]:
import numpy as np
import pandas as pd
import datetime
import requests
import matplotlib.pyplot as plt

api_key = pd.read_json('fmp_api_key.json').iloc[0].to_list()[0]

In [3]:
### Financial Statements
revenuesExpenses = ['revenue',
 'costOfRevenue',
 'grossProfit',
 'researchAndDevelopmentExpenses',
 'generalAndAdministrativeExpenses',
 'sellingAndMarketingExpenses',
 'otherExpenses',                   
 'operatingExpenses',
 'costAndExpenses',
 'interestExpense',
 'depreciationAndAmortization',
 'operatingIncome',
 'totalOtherIncomeExpensesNet',
 'incomeBeforeTax', 
 'incomeTaxExpense',
 'netIncome']

ignoreList = []
headers = {'Content-Type': 'application/json'}

def drop_ignoreList(df, ignoreList):
    """ 
    Input: dataframe and a list of columns to be dropped.
    Output: dataframe with indices 'symbol' and 'date'
    """
    df.drop(ignoreList, axis=1, inplace=True)
    df['date'] = pd.to_datetime(df['date'])
    df.set_index(['symbol', 'date'], inplace=True)
    
    return df

def get_income_statement(ticker, period):
    """ 
    Inputs: stock ticker and period of income_statement ('FY' or 'quarter'). 
            Inputs must be of string type.
    Outputs: pandas dataframe for income statement with revenues and expenses given in millions.  
    """  
    if period == 'FY':
        limit = 120
    else:
        limit = 400
        
    response = requests.get("https://financialmodelingprep.com/api/v3/income-statement/" +
                       ticker + "?period=" + period + "&limit=" + str(limit) + "&apikey=" + api_key, headers=headers)    
    df = pd.json_normalize(response.json())
    df = drop_ignoreList(df, ignoreList)
    
    # Convert revenueExpenses into millions.
    df[revenuesExpenses] = df[revenuesExpenses].apply(lambda x:x*1.0e-6)
    
    return df

def get_balance_sheet_statement(ticker, period):
    """ 
    Inputs: stock ticker and period of balance-sheet-statement ('FY' or 'quarter'). 
            Inputs must be of string type.
    Outputs: pandas dataframe for balance-sheet-statement in $millions.  
    """   
    if period == 'FY':
        limit = 120
    else:
        limit = 400
    response = requests.get("https://financialmodelingprep.com/api/v3/balance-sheet-statement/" +
                       ticker + "?period=" + period + "&limit=" + str(limit) + "&apikey=" + api_key, headers=headers)
    df = pd.json_normalize(response.json())
    df = drop_ignoreList(df, ignoreList)
    
    # Convert values into millions.
    df = df.apply(lambda x:x*1.0e-6)
    
    return df

def get_cash_flow_statement(ticker, period):
    """ 
    Inputs: stock ticker and period of cash-flow-statement ('FY' or 'quarter'). 
            Inputs must be of string type.
    Outputs: pandas dataframe for cash-flow-statement in $millions.  
    """   
    if period == 'FY':
        limit = 120
    else:
        limit = 400
    response = requests.get("https://financialmodelingprep.com/api/v3/cash-flow-statement/" +
                       ticker + "?period=" + period + "&limit=" + str(limit) + "&apikey=" + api_key, headers=headers)
    df = pd.json_normalize(response.json())
    df = drop_ignoreList(df, ignoreList)
    
    # Convert values into millions.
    df = df.apply(lambda x:x*1.0e-6)
    
    return df

### Financial Statements Growth
def get_income_statement_growth(ticker, period='FY'):
    """ 
    Inputs: stock ticker and period of income-statement-growth ('FY' only). 
            Inputs must be of string type.
    Outputs: pandas dataframe for income-statement-growth. 
    """
    response = requests.get("https://financialmodelingprep.com/api/v3/income-statement-growth/" +
                       ticker + "?period=" + period + "&apikey=" + api_key, headers=headers)    
    df = pd.json_normalize(response.json())
    df = drop_ignoreList(df, [])
    
    return df

def get_balance_sheet_statement_growth(ticker, period='FY'):
    """ 
    Inputs: stock ticker and period of balance-sheet-statement-growth ('FY' or 'quarter'). 
            Inputs must be of string type.
    Outputs: pandas dataframe for balance-sheet-statement-growth.
    """   
    response = requests.get("https://financialmodelingprep.com/api/v3/balance-sheet-statement-growth/" +
                       ticker + "?period=" + period + "&apikey=" + api_key, headers=headers)
    df = pd.json_normalize(response.json())
    df = drop_ignoreList(df, [])
    
    return df

def get_cash_flow_statement_growth(ticker, period='FY'):
    """ 
    Inputs: stock ticker and period of cash-flow-statement-growth ('FY' or 'quarter'). 
            Inputs must be of string type.
    Outputs: pandas dataframe for cash-flow-statement-growth.
    """   
    response = requests.get("https://financialmodelingprep.com/api/v3/cash-flow-statement-growth/" +
                       ticker + "?period=" + period + "&apikey=" + api_key, headers=headers)
    df = pd.json_normalize(response.json())
    df = drop_ignoreList(df, [])
    
    return df

### Company Financial Ratios
def get_financial_ratios_ttm(ticker):
    """ 
    Inputs: stock ticker (type: string)
    Outputs: pandas dataframe for company TTM ratios
    """   
    response = requests.get("https://financialmodelingprep.com/api/v3/ratios-ttm/" +
                       ticker + "?apikey=" + api_key, headers=headers)
    df = pd.json_normalize(response.json())
    
    return df

def get_financial_ratios(ticker, period):
    """ 
    Inputs: stock ticker and period ('FY' or 'quarter')
            Inputs must be of string type.
    Outputs: pandas dataframe for company ratios
    """   
    if period == 'FY':
        limit = 40
    else:
        limit = 140
    response = requests.get("https://financialmodelingprep.com/api/v3/ratios/" +
                       ticker + "?period=" + period + "&limit=" + str(limit) + "&apikey=" + api_key, headers=headers)
    df = pd.json_normalize(response.json())
    df = drop_ignoreList(df, [])
    
    return df


### Company Key Metrics
def get_key_metrics_ttm(ticker):
    """ 
    Inputs: stock ticker (type: string)
    Outputs: pandas dataframe for company TTM key metrics
    """   
    response = requests.get("https://financialmodelingprep.com/api/v3/key-metrics-ttm/" +
                       ticker + "?limit=40" + "&apikey=" + api_key, headers=headers)
    if response.json() != []:
        df = pd.json_normalize(response.json())        
        return df
    
    else:
        pass
    
def get_key_metrics(ticker, period):
    """ 
    Inputs: stock ticker and period ('FY' or 'quarter')
            Inputs must be of string type.
    Outputs: pandas dataframe for company key metrics
    """   
    if period == 'FY':
        limit = 40
    else:
        limit = 130
        
    response = requests.get("https://financialmodelingprep.com/api/v3/key-metrics/" +
                       ticker + "?period=" + period + "&limit=" + str(limit) + "&apikey=" + api_key, headers=headers)
    if response.json() != []:
        df = pd.json_normalize(response.json())
        df = drop_ignoreList(df, [])
        
        return df
    
    else:
        pass

### Company Financial Growth
def get_financial_growth(ticker, period):
    """ 
    Inputs: stock ticker and period ('FY' or 'quarter')
            Inputs must be of string type.
    Outputs: pandas dataframe for company financial growth
    """   
    if period == 'FY':
        limit = 20
    else:
        limit = 80
    response = requests.get("https://financialmodelingprep.com/api/v3/financial-growth/" +
                       ticker + "?period=" + period + "&limit=" + str(limit) + "&apikey=" + api_key, headers=headers)
    df = pd.json_normalize(response.json())
    df = drop_ignoreList(df, [])
    
    return df

### Company Discounted cash flow value
def get_discounted_cash_flow(ticker):
    """ 
    Inputs: stock ticker and period ('FY' or 'quarter')
            Inputs must be of string type.
    Outputs: pandas dataframe for company discounted cash flow
    """   
    response = requests.get("https://financialmodelingprep.com/api/v3/discounted-cash-flow/" +
                       ticker + "?apikey=" + api_key, headers=headers)
    df = pd.json_normalize(response.json())
    df = drop_ignoreList(df, [])
    
    return df

def get_hist_discounted_cash_flow(ticker, period):
    """ 
    Inputs: stock ticker and period ('FY' or 'quarter')
            Inputs must be of string type.
    Outputs: pandas dataframe for company historical discounted cash flow
    """   
    response = requests.get("https://financialmodelingprep.com/api/v3/historical-discounted-cash-flow/" +
                       ticker + "?period=" + period + "&apikey=" + api_key, headers=headers)
    df = pd.json_normalize(response.json()[0]['historicalDCF'])
    df['date'] = pd.to_datetime(df['date'])
    df.set_index(['date'], inplace=True)
        
    return df

def get_hist_discounted_cash_flow_daily(ticker):
    """ 
    Inputs: stock ticker and period ('FY' or 'quarter')
            Inputs must be of string type.
    Outputs: pandas dataframe for company daily historical discounted cash flow
    """   
    response = requests.get("https://financialmodelingprep.com/api/v3/historical-daily-discounted-cash-flow/" +
                       ticker + "?apikey=" + api_key, headers=headers)
    df = pd.json_normalize(response.json())
    df = drop_ignoreList(df, [])
    
    return df

# Earnings Calendar
def get_earnings_calendar():
    """ 
    Inputs: none
    Outputs: dataframe for earnings calendar
    """   
    response = requests.get("https://financialmodelingprep.com/api/v3/earning_calendar/" +
                       "?apikey=" + api_key, headers=headers)
    df = pd.json_normalize(response.json())
    df = drop_ignoreList(df, [])
    
    return df

# Company Profile
def get_profile(ticker):
    """
    Inputs: stock ticker in string type
    Outputs: pandas dataframe for company profile
    """
    response = requests.get("https://financialmodelingprep.com/api/v3/profile/" +
                       ticker + "?apikey=" + api_key, headers=headers)
    if response.json() != []:
        df = pd.json_normalize(response.json())
        
        return df.transpose()
    
    else:
        pass

# Institutional holders
def get_institutional_holder(ticker):
    """
    Inputs: stock ticker in string type
    Outputs: dataframe for institutional holders
    """
    response = requests.get("https://financialmodelingprep.com/api/v3/institutional-holder/" +
                       ticker + "?apikey=" + api_key, headers=headers)
    df = pd.json_normalize(response.json())    
    
    return df

# Earnings Surprises
def get_earnings_surprises(ticker):
    """
    Inputs: stock ticker in string type
    Outputs: pandas dataframe for company earning-surprises
    """
    response = requests.get("https://financialmodelingprep.com/api/v3/earnings-surprises/" +
                       ticker + "?apikey=" + api_key, headers=headers)
    df = pd.json_normalize(response.json())
    df = drop_ignoreList(df, [])
    df['earningSurprise'] =  ((df['actualEarningResult'] - df['estimatedEarning']) / abs(df['estimatedEarning'])) * 100
    
    return df

# Key Executives
def get_key_executives(ticker):
    """
    Inputs: stock ticker in string type
    Outputs: pandas dataframe for company key executives
    """
    response = requests.get("https://financialmodelingprep.com/api/v3/key-executives/" +
                       ticker + "?apikey=" + api_key, headers=headers)
    df = pd.json_normalize(response.json())
    
    return df

### Stock Screener
- marketCapMoreThan & marketCapLowerThan : Number
- priceMoreThan & priceLowerThan : Number
- betaMoreThan & betaLowerThan : Number
- volumeMoreThan & volumeLowerThan : Number
- dividendMoreThan & dividendLowerThan : Number
- isEtf & isActivelyTrading : true/false
- sector : Consumer Cyclical - Energy - Technology - Industrials - Financial Services - Basic Materials - Communication Services - Consumer Defensive - Healthcare - Real Estate - Utilities - Industrial Goods - Financial - Services - Conglomerates
- Industry : Autos - Banks - Banks Diversified - Software - Banks Regional - Beverages Alcoholic - Beverages Brewers - Beverages - Non-Alcoholic
- Country : US - UK - MX - BR - RU - HK - CA - ...
- exchange : nyse - nasdaq - amex - euronex - tsx - etf - mutual_fund
- limit : Number

Large Cap >= 10B

2B =< Mid Cap < 10B

0.3B < Small Cap < 2B

In [4]:
profile = get_profile('ATKR').iloc[[0,4,9,14,15,19,20,17]]
profile

,0
symbol,ATKR
mktCap,3268412416
currency,USD
exchangeShortName,NYSE
industry,Specialty Industrial Machinery
sector,Industrials
country,US
description,Atkore International Group Inc. manufactures a...


In [5]:
profile.loc['mktCap']/1.0e9

0    3.268412
Name: mktCap, dtype: object

In [6]:
large_cap = 10000000000
mid_cap = 2000000000
small_cap = 300000000

params = {'sector': 'Industrials',
          'industry': 'Specialty Industrial Machinery',
          'country': 'US',
          #'exchange': 'NASDAQ',
          'isEtf': False,
          'marketCapLowerThan': large_cap,
          'marketCapMoreThan': mid_cap,
          'volumeMoreThan': 100000,
          'limit': 300}

def get_stock_screener(params):
    response = requests.get("https://financialmodelingprep.com/api/v3/stock-screener?" 
                            + 'marketCapLowerThan=' + str(params['marketCapLowerThan'])
                            + '&marketCapMoreThan=' + str(params['marketCapMoreThan'])
                            + '&volumeMoreThan=' + str(params['volumeMoreThan'])
                            + '&sector=' + str(params['sector'])
                            + '&industry=' + str(params['industry'])
                            + '&country=' + str(params['country'])
                            #+ '&exchange=' + str(params['exchange'])
                            + '&isEtf=' + str(params['isEtf'])
                            + '&limit=' + str(params['limit'])
                            + "&apikey=" + api_key, headers=headers)
    df = pd.json_normalize(response.json())
    df.set_index(['symbol'], inplace=True)
    
    return df

### Initial screening by sector and market cap

In [7]:
screened_tickers = get_stock_screener(params)
tickers = screened_tickers.index.to_list()
print(tickers[:])
print('\n')
print("number of screened tickers =", len(tickers))

['MIDD', 'ITT', 'DCI', 'CFX', 'RBC', 'RXN', 'CR', 'GTLS', 'FLS', 'CW', 'GTES', 'JBT', 'PSN', 'WTS', 'AIMC', 'FELE', 'HI', 'ATKR', 'ARNC', 'FLOW', 'SPXC', 'B', 'MWA', 'HLIO', 'WBT', 'TPIC']


number of screened tickers = 26


### Industries

In [ ]:
# find industries of the screened stock tickers
def get_industries_from_profiles(tickers):
    
    temp = []

    for ticker in tickers:
        df_industry = get_profile(ticker).loc[['symbol', 'industry']]
        temp.append(df_industry)
    
    df = pd.concat(temp, axis=1)
    df = df.transpose()
    df.columns = ['symbol', 'industry']
    df.reset_index(drop=True, inplace=True)
    
    return df

In [ ]:
df_industry = get_industries_from_profiles(tickers)
df_industry['industry'].value_counts()

In [ ]:
df_industry['industry'].value_counts().plot.bar();

### Key Metrics

In [8]:
def get_key_metrics_tickers(tickers, period):
    """
    Input: a list of stock tickers and period ('FY', or 'quarter')
    Output: pandas dataframe for selected key metrics
    """ 
    metrics = [
        'peRatio',
        'enterpriseValueOverEBITDA',
        'evToSales',
        'freeCashFlowYield',
        'debtToEquity',
        'roic',
        'roe',
        'salesGeneralAndAdministrativeToRevenue',
        'researchAndDdevelopementToRevenue'
    ]
    
    dataframes = []
    
    for ticker in (tickers):
        df = get_key_metrics(ticker, period)
        dataframes.append(df)
    df = pd.concat(dataframes)
    df['peRatio'] = pd.to_numeric(df['peRatio'])
    df['enterpriseValueOverEBITDA'] = pd.to_numeric(df['enterpriseValueOverEBITDA'])
    df['evToSales'] = pd.to_numeric(df['evToSales'])
    df['freeCashFlowYield'] = pd.to_numeric(df['freeCashFlowYield'])
    df['debtToEquity'] = pd.to_numeric(df['debtToEquity'])
    df['roic'] = pd.to_numeric(df['roic'])
    df['roe'] = pd.to_numeric(df['roe'])
    df['salesGeneralAndAdministrativeToRevenue'] = pd.to_numeric(df['salesGeneralAndAdministrativeToRevenue'])
    df['researchAndDdevelopementToRevenue'] = pd.to_numeric(df['researchAndDdevelopementToRevenue'])
    df_metrics = df[metrics]
    df_metrics = df_metrics.rename(columns={'enterpriseValueOverEBITDA': 'EV/EBITDA', 
                       'salesGeneralAndAdministrativeToRevenue': 'SG&A/revenue', 
                       'researchAndDdevelopementToRevenue': 'R&D/revenue' })
    
    return df_metrics


def make_comparison_plot(df_0, ticker_a, ticker_b, metrics):
    
    df_a = df_0.loc[ticker_a][metrics][:20].sort_index()
    df_b = df_0.loc[ticker_b][metrics][:20].sort_index()
    df = pd.concat([df_a, df_b], axis=1)
    df.columns = [metrics + '_' + ticker_a, metrics + '_' + ticker_b]
    ax = df.plot.bar(figsize=(10,7), grid=True)
    ax.set_xlabel('')
    ax.set_ylabel(metrics, fontsize=14)
    plt.tight_layout()
    
    return ax   

### Tickers selection
get quarterly key metrics for given sector and market cap

In [9]:
df_metrics_quarter = get_key_metrics_tickers(tickers, 'quarter')
#df_metrics_quarter.head(2)

In [10]:
tickers_selected = df_metrics_quarter.index.get_level_values(0).unique().to_list()
print("number of selected tickers =", len(tickers_selected))

number of selected tickers = 26


In [ ]:
df_last4q = df_metrics_quarter.groupby('symbol').head(4)
pe_25 = df_last4q['peRatio'].groupby('symbol').min().describe()['25%']
pe_75 = df_last4q['peRatio'].groupby('symbol').max().describe()['75%']
print("pe_25=", pe_25)
print("pe_75=", pe_75)

In [ ]:
df_last4q[['peRatio']].describe().transpose()

In [ ]:
fig, ax = plt.subplots(figsize=(5,5))
df_last4q['peRatio'].hist(bins=100)
#ax.set_xlim(-3000, 10000)

In [ ]:
df_last4q[df_last4q['peRatio'] > pe_75].index.get_level_values(0).nunique()

In [ ]:
tickers_ignored = set(df_last4q[(df_last4q['peRatio'] <= pe_25) | (df_last4q['peRatio'] >= pe_75)].index.get_level_values(0).unique())
all_tickers = set(df_metrics_quarter.index.get_level_values(0).unique())
tickers_selected = list(all_tickers - tickers_ignored)
print(tickers_selected)
print('\n')
print("number of selected tickers =", len(tickers_selected))

In [ ]:
'RMD' in tickers_selected

In [11]:
# Selected tickers' statistic
quarterly_stat = df_metrics_quarter.loc[tickers_selected].describe().transpose()
quarterly_stat

,count,mean,std,min,25%,50%,75%,max
peRatio,1870.0,17.768169,134.417279,-1285.044124,10.519776,15.804643,22.372573,5005.166435
EV/EBITDA,1873.0,8.099598,1243.352743,-53676.812778,25.982901,35.111325,46.185339,1325.750000
evToSales,1873.0,5.449560,3.038485,-11.027583,3.460747,4.927134,7.088044,22.906068
freeCashFlowYield,1871.0,0.017293,0.104743,-2.340943,0.002174,0.014800,0.029596,1.003922
debtToEquity,1870.0,0.719603,3.574650,-51.460076,0.292608,0.506253,0.839283,117.601770
roic,1873.0,0.049946,1.491271,-52.496144,0.021376,0.032068,0.046372,27.147860
roe,1870.0,0.061211,3.689857,-135.340517,0.021015,0.035492,0.053614,57.357143
SG&A/revenue,1873.0,-0.215127,17.555605,-212.336283,0.120250,0.185336,0.220248,582.148992
R&D/revenue,1873.0,0.005087,0.014644,-0.198420,0.000000,0.000000,0.000000,0.113217


### Get key metrics for selected tickers
FY

In [12]:
df_metrics = get_key_metrics_tickers(tickers_selected, 'FY')
df_metrics.head()

peRatio  EV/EBITDA  evToSales  freeCashFlowYield  \
symbol date                                                             
MIDD   2021-01-02  36.698166  20.849595   3.608386           0.064403   
       2019-12-28  17.828083  13.045973   2.721966           0.052680   
       2018-12-29  20.341231  15.221646   3.036602           0.050761   
       2017-12-30  25.864633  19.280330   3.701517           0.032416   
       2016-12-31  27.004452  16.192260   3.676932           0.035087   

                   debtToEquity      roic       roe  SG&A/revenue  R&D/revenue  
symbol date                                                                     
MIDD   2021-01-02      0.011608  0.072060  0.104871      0.211637          0.0  
       2019-12-28      0.960670  0.116589  0.180932      0.197271          0.0  
       2018-12-29      1.134335  0.111748  0.190458      0.197890          0.0  
       2017-12-30      0.752109  0.132102  0.219027      0.200476          0.0  
       2016-12-31      0.573961  0.188061  0.224620      0.097250          0.0

In [13]:
df_metrics.dropna().index.get_level_values(0).nunique()

26

In [14]:
FY_stat = df_metrics.dropna().describe().transpose()
FY_stat

,count,mean,std,min,25%,50%,75%,max
peRatio,565.0,21.995433,113.051541,-605.750025,7.712181,15.500537,23.220764,2225.944645
EV/EBITDA,565.0,9.073755,21.172877,-336.404233,5.732612,8.545577,11.755204,321.061065
evToSales,565.0,1.259713,0.799335,-0.056862,0.715184,1.104579,1.677570,5.607276
freeCashFlowYield,565.0,1.254711,26.341521,-6.780580,0.034556,0.058756,0.097801,625.248905
debtToEquity,565.0,2.311967,35.110692,-30.205257,0.326081,0.572353,0.960670,832.000000
roic,565.0,0.132086,0.097905,-0.507685,0.082911,0.120715,0.178152,1.258249
roe,565.0,0.084718,0.882776,-19.000000,0.075865,0.130468,0.188818,3.277778
SG&A/revenue,565.0,-0.183138,7.726543,-183.501524,0.000000,0.175268,0.211840,0.354114
R&D/revenue,565.0,0.007193,0.016355,0.000000,0.000000,0.000000,0.009165,0.213468


In [15]:
FY_stat['50%'].to_frame().transpose()

,peRatio,EV/EBITDA,evToSales,freeCashFlowYield,debtToEquity,roic,roe,SG&A/revenue,R&D/revenue
50%,15.500537,8.545577,1.104579,0.058756,0.572353,0.120715,0.130468,0.175268,0.0


In [16]:
df_metrics.dropna().groupby(level=0).median().sort_values(by=['peRatio', 'EV/EBITDA', 'evToSales', 'debtToEquity'])

,peRatio,EV/EBITDA,evToSales,freeCashFlowYield,debtToEquity,roic,roe,SG&A/revenue,R&D/revenue
symbol,,,,,,,,,
SPXC,2.429027,4.214836,0.363144,0.138365,0.809741,0.078920,0.093126,0.189982,0.000000
ITT,2.896561,2.475292,0.319106,0.100624,0.182126,0.129507,0.186314,0.143898,0.032667
MIDD,10.783585,7.297618,0.852045,0.072563,0.871493,0.141981,0.182259,0.101062,0.000000
FLOW,11.133416,15.032207,1.181640,0.075140,0.804383,0.068729,0.046180,0.234319,0.000000
ATKR,12.037801,9.338778,1.063269,0.104449,2.445309,0.170927,0.402479,0.124331,0.000000
B,13.649358,8.239667,0.828183,0.068825,0.696879,0.105886,0.130396,0.260504,0.000000
ARNC,14.383196,6.638857,0.804604,0.064146,1.885221,0.065150,0.066798,0.000000,0.009070
CW,14.703163,8.864571,1.417028,0.057345,0.422350,0.111135,0.114566,0.177626,0.026555
CR,14.761478,7.841586,1.003823,0.069194,0.489577,0.143905,0.180590,0.206714,0.000000


In [17]:
df_metrics.dropna().groupby(level=0).mean().sort_values(by=['freeCashFlowYield', 'roic', 'roe'], ascending=False).head()

,peRatio,EV/EBITDA,evToSales,freeCashFlowYield,debtToEquity,roic,roe,SG&A/revenue,R&D/revenue
symbol,,,,,,,,,
DCI,17.728283,9.925211,1.423071,18.246970,0.345348,0.229634,0.211281,0.159310,0.021584
FLOW,63.374012,12.567172,1.096117,0.260518,0.867348,0.073433,-0.056687,0.204296,0.000000
SPXC,2.296856,13.491614,0.491906,0.206578,0.924694,0.076359,0.073587,0.141620,0.000000
FELE,16.512185,8.422958,1.254759,0.181044,0.377846,0.192147,0.174846,-4.577341,0.008462
MIDD,11.568642,0.769821,1.423778,0.174938,28.561668,0.149267,-0.372456,0.116095,0.000000


In [18]:
df_metrics.dropna().groupby(level=0).median().sort_values(by=['debtToEquity'], ascending=False).head(10)

,peRatio,EV/EBITDA,evToSales,freeCashFlowYield,debtToEquity,roic,roe,SG&A/revenue,R&D/revenue
symbol,,,,,,,,,
WBT,29.138520,15.900826,2.414526,-0.051624,6.347157,0.138214,0.320238,0.196965,0.000000
ATKR,12.037801,9.338778,1.063269,0.104449,2.445309,0.170927,0.402479,0.124331,0.000000
ARNC,14.383196,6.638857,0.804604,0.064146,1.885221,0.065150,0.066798,0.000000,0.009070
RXN,34.902390,12.551836,1.907731,0.065711,1.500280,0.092995,0.069214,0.203890,0.000000
JBT,24.214581,11.530535,1.051925,0.045265,1.458054,0.158906,0.258389,0.175872,0.016436
GTES,24.515460,19.928439,2.157228,0.051777,1.307840,0.034571,0.137543,0.246250,0.000000
MWA,22.705667,10.730286,1.535681,0.047408,1.145589,0.089916,0.084357,0.186260,0.000000
AIMC,21.140321,8.171192,1.094018,0.072586,1.027792,0.121719,0.104706,0.180216,0.017358
HI,16.248304,10.603080,1.613638,0.078462,0.873085,0.147629,0.187511,0.000000,0.000000


### Check interesting stocks
quarterly key metrics

In [ ]:
ticker_of_interest = 'RBC'

In [ ]:
profile = get_profile(ticker_of_interest)
profile.iloc[[0, 4, 9, 14, 15, 17, 19, 20,]]

In [ ]:
df_stock_metrics = df_metrics_quarter.loc[ticker_of_interest]
df_stock_metrics.head()

In [ ]:
quarterly_stat.loc[['peRatio', 'EV/EBITDA']]

In [ ]:
df_stock_metrics[['peRatio', 'EV/EBITDA']][:20].sort_index().plot.bar(grid=True, figsize=(10,7));
plt.axhline(y = quarterly_stat.loc['peRatio']['50%'], lw=1.2,ls='--', color='b');
plt.axhline(y = quarterly_stat.loc['EV/EBITDA']['50%'], lw=1.2,ls='--', color='r');

In [ ]:
quarterly_stat.loc[['evToSales']]

In [ ]:
df_stock_metrics['evToSales'][:20].sort_index().plot.bar(grid=True, figsize=(10,7), legend=True);
plt.axhline(y = quarterly_stat.loc['evToSales']['50%'], lw=1.2,ls='--', color='m');

In [ ]:
quarterly_stat.loc[['freeCashFlowYield']]

In [ ]:
df_stock_metrics['freeCashFlowYield'][:20].sort_index().plot.bar(grid=True, figsize=(10,7), legend=True);
plt.axhline(y = quarterly_stat.loc['freeCashFlowYield']['50%'], lw=1.2,ls='--', color='m');

In [ ]:
quarterly_stat.loc[['roic', 'roe']]

In [ ]:
df_stock_metrics[['roic', 'roe']][:20].sort_index().plot.bar(grid=True, figsize=(10,7), legend=True);
plt.axhline(y = quarterly_stat.loc['roic']['50%'], lw=1.2,ls='--', color='b');
plt.axhline(y = quarterly_stat.loc['roe']['50%'], lw=1.2,ls='--', color='r');

In [ ]:
quarterly_stat.loc[['debtToEquity']]

In [ ]:
df_stock_metrics['debtToEquity'][:20].sort_index().plot.bar(grid=True, figsize=(10,7), legend=True);
plt.axhline(y = quarterly_stat.loc['debtToEquity']['50%'], lw=1.2,ls='--', color='m');

### Income Statement

In [ ]:
df_income = get_income_statement(ticker_of_interest, 'FY')
df_income.head()

In [ ]:
df_income.loc[ticker_of_interest][['revenue', 'grossProfit', 'netIncome']].sort_index().plot.bar(figsize = (10,7));
plt.tight_layout()

In [ ]:
df_income.loc[ticker_of_interest][['netIncome']].sort_index().plot.bar(figsize = (10,7));
plt.tight_layout()

In [ ]:
df_income.loc[ticker_of_interest][['ebitda']].sort_index().plot.bar(figsize = (10,7));
plt.tight_layout()

In [ ]:
df_income.loc[ticker_of_interest][['eps']].sort_index().plot.bar(figsize = (10,7));
plt.tight_layout()

### Cashflow Statement

In [ ]:
df_cashflow = get_cash_flow_statement(ticker_of_interest, 'FY')
df_cashflow.head()

In [ ]:
df_cashflow.loc[ticker_of_interest][['freeCashFlow']].sort_index().plot.bar(figsize = (10,7));
plt.tight_layout()

### Comparison
check for both FY and quarter

In [ ]:
metrics = df_metrics.columns.to_list()

for metric in metrics:
    make_comparison_plot(df_metrics_quarter, 'CW', 'RBC', metric);

In [ ]:
earning_calender = get_earnings_calendar()

In [ ]:
earning_calender.loc['RUN']